In [1]:
#Import necessary libraries

import numpy as np
import pandas as pd
import seal
from seal import *
import time
import pickle

In [2]:
#Data owner reloads encryption context

parms = EncryptionParameters(scheme_type.ckks)
parms.load('2/public/IDASH_parms')

context = SEALContext(parms)
ckks_encoder = CKKSEncoder(context)

In [3]:
keygen = KeyGenerator(context)
secret_key = keygen.secret_key()
secret_key.load(context, '2/private/IDASH_secretkey')

public_key = keygen.create_public_key()
public_key.load(context, '2/public/IDASH_pubkey')

scale = pickle.load(open('2/public/IDASH_scale','rb'))

In [4]:
encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)

In [5]:
#Initialize and load ciphertexts from Model Owner, Step 3

results = []
for i in range(3):
    pt_init = ckks_encoder.encode(0.,scale)
    ct_init = encryptor.encrypt(pt_init)
    ct_init.load(context, '3/public/IDASH_ct_results_%s' % i)
    results.append(ct_init)

In [6]:
decryptor = Decryptor(context, secret_key)

In [7]:
#Decrypt and decode results

final_scores = []
for i in range(3):
    pt_final = decryptor.decrypt(results[i])
    final_scores.append(ckks_encoder.decode(pt_final))

In [8]:
#DATA OWNER postprocessing of results
#The first 4 entries of scores are the correct entries of the matrix-vector product. 
final_junk_removed = []
for j in range(1000):
    final_junk_removed.append(final_scores[j // 341][24*(j%341):24*(j%341)+4])
    
final_junk_removed

[array([-0.00221687,  0.15851439,  0.18209602,  0.39449925]),
 array([0.10965798, 0.01617647, 0.00308116, 0.96590364]),
 array([ 0.44304905, -0.00443877, -0.004918  ,  1.05585363]),
 array([0.0011397 , 1.00579192, 0.00324338, 0.20329861]),
 array([7.61514942e-04, 3.26761157e-02, 1.06883155e+00, 4.34351862e-02]),
 array([8.69748722e-04, 1.04413848e+00, 2.06664127e-03, 1.89494378e-01]),
 array([ 0.74305026, -0.00491493,  0.00249491,  1.05053241]),
 array([ 7.89258252e-04,  1.04515208e+00, -1.43766109e-04,  2.13252840e-01]),
 array([-0.00508467,  0.01709696,  0.96549598,  0.02170925]),
 array([0.00114103, 1.00023458, 0.00433574, 0.19757774]),
 array([-0.00305881,  0.15357664,  0.16901631,  0.38565278]),
 array([-0.00322792,  0.02892924,  1.05564168,  0.06916398]),
 array([-1.02833318e-03,  2.87245091e-02,  1.05617084e+00,  3.43359645e-02]),
 array([0.00108269, 1.00841964, 0.00145402, 0.23194665]),
 array([ 0.51584377, -0.00466548, -0.00485603,  1.06460299]),
 array([0.00108651, 1.00889917

In [9]:
#DATA OWNER postprocessing
#Very simple function to convert final_junk_removed to list of label predictions
#Correct conversion assumes all scores in final_junk_removed round to 0 or 1
#Could update function to change scores >1.5 to 0 and <-.5 to 1
#This version miscategorizes all 0's as belonging to the 0th category
#and if more than one category has a 1, the reported category is the sum.
def conv_to_pred(score_array):
    predictions = []
    for scores in score_array:
        predictions.append(int(np.dot(np.round(scores),np.array([0,1,2,3]))))
        
    return predictions

In [10]:
from sklearn.metrics import classification_report,confusion_matrix

In [11]:
test_labels = pd.read_pickle('1/public/data_owner_labels')

In [12]:
pred = conv_to_pred(final_junk_removed)

In [13]:
#21 samples classified incorrectly, for 2% error.
#Likely all error coming from application of threshold polynomial.

print(confusion_matrix(pred,test_labels))

[[237   0   0   0   0]
 [  0 236   0   0   0]
 [  0   0 248   0   0]
 [ 19   0   0 258   0]
 [  0   0   2   0   0]]


In [14]:
#DATA OWNER postprocessing
#Very simple function to convert final_junk_removed to list of label predictions
#Just takes index of maximum score
def conv_to_pred2(score_array):
    predictions = []
    for scores in score_array:
        predictions.append(np.argmax(scores))
        
    return predictions

In [15]:
#This method fails for every sample in category 0
#but works for the other categories.
#Must figure out why.
#But for now use conv_to_pred

pred2 = conv_to_pred2(final_junk_removed)
print(confusion_matrix(pred2, test_labels))

[[  0   0   0   0]
 [  0 236   0   0]
 [  0   0 249   0]
 [256   0   1 258]]
